Gerekli paketleri yükleyin.

In [ ]:
!pip install -q rapidfuzz gradio joblib
!pip install -q scikit-learn pandas

Veri setini indirin.

In [ ]:
url = "https://huggingface.co/api/resolve-cache/datasets/abdbali/mevsimler_iklim_veriseti/e2eb567a8b6ed9fef709881f7647c87bdabfbc8c/mevsimler_iklim_veriseti.csv?%2Fdatasets%2Fabdbali%2Fmevsimler_iklim_veriseti%2Fresolve%2Fmain%2Fmevsimler_iklim_veriseti.csv=&etag=%2275682dcf069517913339aa3d59120b8e740901ac%22"
!wget -q -O mevsimler_iklim_veriseti.csv "$url"
!ls -l mevsimler_iklim_veriseti.csv


-rw-r--r-- 1 root root 6132 Sep 11 06:33 mevsimler_iklim_veriseti.csv


Veri yükleme ve verileri kontrol edin.

In [ ]:

import pandas as pd
df = pd.read_csv('mevsimler_iklim_veriseti.csv')
df.columns = [c.strip().lower() for c in df.columns]
print("Sütunlar:", df.columns.tolist())
if 'input' not in df.columns or 'output' not in df.columns:
    print("Uyarı: beklenen 'input' ve 'output' sütunları yok. Mevcut sütunlar yukarıda listelendi.")

df.head(5)


Sütunlar: ['input', 'output']


,input,output
0,21 Mart’ta kuzey yarım kürede hangi mevsim baş...,İlkbahar
1,21 Mart’ta güney yarım kürede hangi mevsim baş...,Sonbahar
2,21 Mart’ta gece ve gündüz süreleri nasıldır?,"Eşittir, 12 saat"
3,21 Mart’ta güneş ışınları nereye dik düşer?,Ekvatora
4,21 Mart’ta eksen eğikliği etkisi nasıldır?,Ortadan kalkar


Basit normalize ve tokenizasyon fonksiyonlarını oluşturun.

In [ ]:

import re

def normalize_text(s: str) -> str:
    s = str(s).lower()
    # Türkçe karakterleri koruyarak, diğer noktalama işaretlerini temizle
    s = re.sub(r"[^a-z0-9ığüşöçİĞÜŞÖÇ\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


df['input_norm'] = df['input'].astype(str).apply(normalize_text)
df['output'] = df['output'].astype(str)
df.head(6)


,input,output,input_norm
0,21 Mart’ta kuzey yarım kürede hangi mevsim baş...,İlkbahar,21 mart ta kuzey yarım kürede hangi mevsim başlar
1,21 Mart’ta güney yarım kürede hangi mevsim baş...,Sonbahar,21 mart ta güney yarım kürede hangi mevsim başlar
2,21 Mart’ta gece ve gündüz süreleri nasıldır?,"Eşittir, 12 saat",21 mart ta gece ve gündüz süreleri nasıldır
3,21 Mart’ta güneş ışınları nereye dik düşer?,Ekvatora,21 mart ta güneş ışınları nereye dik düşer
4,21 Mart’ta eksen eğikliği etkisi nasıldır?,Ortadan kalkar,21 mart ta eksen eğikliği etkisi nasıldır
5,21 Haziran’da kuzey yarım kürede hangi mevsim ...,Yaz,21 haziran da kuzey yarım kürede hangi mevsim ...


TF-IDF matrisi hazırlayın (hızlı semantik geri dönüş için)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))  # unigram+bigram
X_tfidf = vectorizer.fit_transform(df['input_norm'].tolist())
print("TF-IDF hazır. Satır (doküman) sayısı:", X_tfidf.shape[0])


TF-IDF hazır. Satır (doküman) sayısı: 82


Eşleşme / cevap fonksiyonlarını oluşturun.

In [ ]:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import process, fuzz

# Parametreler (ihtiyaca göre değiştir)
MIN_COMMON_WORDS = 1
OVERLAP_RATIO_THRESHOLD = 0.30   # ortak kelime / toplam soru kelimesi
TFIDF_SIM_THRESHOLD = 0.35
FUZZY_THRESHOLD = 70  # 0-100 arası rapidfuzz skoru

def get_answer(user_query: str):
    q_norm = normalize_text(user_query)
    # 1) Tam eşleşme (case-insensitive, normalize edilmiş)
    mask = df['input_norm'] == q_norm
    if mask.any():
        idx = df[mask].index[0]
        return df.at[idx, 'output'], {'method': 'exact', 'matched_input': df.at[idx, 'input'], 'score': 1.0}

    # 2) Containment (kısmî içerme: dataset inputi query içinde veya query dataset input içinde)
    for idx, row in df[['input_norm', 'input', 'output']].iterrows():
        if row['input_norm'] in q_norm or q_norm in row['input_norm']:
            return row['output'], {'method': 'containment', 'matched_input': row['input'], 'score': 0.9}

    # 3) Ortak kelime (overlap)
    q_tokens = set([t for t in q_norm.split() if t])
    if len(q_tokens) > 0:
        overlaps = df['input_norm'].apply(lambda s: len(q_tokens & set(s.split())))
        best_overlap = int(overlaps.max())
        best_idx = int(overlaps.idxmax())
        ratio = best_overlap / max(1, len(q_tokens))
        if best_overlap >= MIN_COMMON_WORDS and ratio >= OVERLAP_RATIO_THRESHOLD:
            return df.at[best_idx, 'output'], {'method': 'overlap', 'matched_input': df.at[best_idx, 'input'], 'score': float(ratio), 'common_words': best_overlap}

    # 4) TF-IDF + cosine similarity
    q_vec = vectorizer.transform([q_norm])
    cosines = cosine_similarity(q_vec, X_tfidf).flatten()
    best_idx = int(np.argmax(cosines))
    best_sim = float(cosines[best_idx])
    if best_sim >= TFIDF_SIM_THRESHOLD:
        return df.at[best_idx, 'output'], {'method': 'tfidf_cosine', 'matched_input': df.at[best_idx, 'input'], 'score': best_sim}

    # 5) Fuzzy fallback (rapidfuzz)
    choices = df['input_norm'].tolist()
    best = process.extractOne(q_norm, choices, scorer=fuzz.token_sort_ratio)
    # best -> (matched_text, score, index)
    if best and best[1] >= FUZZY_THRESHOLD:
        matched_norm = best[0]
        idx = choices.index(matched_norm)
        return df.at[idx, 'output'], {'method': 'fuzzy', 'matched_input': df.at[idx, 'input'], 'score': best[1]}

    # 6) Eğer hiçbiri yoksa - fallback
    # Ayrıca benzer öneriler sunabiliriz (top3 TF-IDF)
    top_indices = np.argsort(-cosines)[:3]
    suggestions = df.iloc[top_indices]['input'].tolist()
    return ("Üzgünüm, bunu anlayamadım. Aşağıdaki benzer sorulardan birini deneyebilirsin:\n- " + "\n- ".join(suggestions),
            {'method': 'no_match', 'suggestions': suggestions})


Hızlı etkileşim

In [ ]:

print("Soru-cevap döngüsü. Çıkmak için 'q' yaz.")
while True:
    q = input("Soru: ")
    if q.strip().lower() in ('q','quit','exit'):
        print("Çıkılıyor.")
        break
    ans, meta = get_answer(q)
    print("\nCevap:")
    print(ans)
    print("\n(Eşleşme bilgisi):", meta)
    print("-"*40)


Soru-cevap döngüsü. Çıkmak için 'q' yaz.

Cevap:
Yaz

(Eşleşme bilgisi): {'method': 'containment', 'matched_input': '21 Haziran’da kuzey yarım kürede hangi mevsim başlar?', 'score': 0.9}
----------------------------------------

Cevap:
En uzun gündüz yaşanır

(Eşleşme bilgisi): {'method': 'overlap', 'matched_input': '21 Haziran’da kuzey yarım kürede gün süresi nasıldır?', 'score': 0.3333333333333333, 'common_words': 1}
----------------------------------------

Cevap:
Eşittir, 12 saat

(Eşleşme bilgisi): {'method': 'overlap', 'matched_input': '21 Mart’ta gece ve gündüz süreleri nasıldır?', 'score': 0.3333333333333333, 'common_words': 1}
----------------------------------------

Cevap:
İlkbahar

(Eşleşme bilgisi): {'method': 'containment', 'matched_input': '21 Mart’ta kuzey yarım kürede hangi mevsim başlar?', 'score': 0.9}
----------------------------------------
